# Классификация текста простыми методами

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TANYA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\TANYA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\TANYA\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\TANYA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
from sklearn.datasets import fetch_20newsgroups

categories = ["comp.sys.mac.hardware", "comp.windows.x",  "misc.forsale", "rec.autos", "rec.motorcycles", "rec.sport.baseball", "rec.sport.hockey", "sci.crypt", "sci.electronics", "sci.med",
              "sci.space", "soc.religion.christian", "talk.politics.guns", "talk.politics.mideast"]
              
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

X_train = newsgroups_train.data
y_train = newsgroups_train.target

X_test = newsgroups_test.data
y_test = newsgroups_test.target

In [3]:
len(X_train), len(y_train)

(8227, 8227)

In [4]:
len(X_test), len(y_test)

(5477, 5477)

## Предобработка данных

In [25]:
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

def preprocces(X):
  X_proccess = []
  lemmatizer = WordNetLemmatizer()

  for x in tqdm(X):
    
    x = x.lower()
    x = nltk.word_tokenize(x)
    x = [word for word in x if word.isalnum()]
    x = [lemmatizer.lemmatize(w) for w in x]
    x = [word for word in x if not word in stop_words]
    X_proccess.append(' '.join(x))
  return X_proccess


X_train_proc = preprocces(X_train)
X_test_proc = preprocces(X_test)

100%|█████████████████████████████████████████████████████████████████████████████| 5477/5477 [00:15<00:00, 363.40it/s]


In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train_proc)
X_test_vec = vectorizer.transform(X_test_proc)
print(X_train_vec.shape)
print(X_test_vec.shape)

(8227, 62970)
(5477, 62970)


*стандартизация результат не улучшила*

<8227x62970 sparse matrix of type '<class 'numpy.float64'>'
	with 852092 stored elements in Compressed Sparse Row format>

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

lr = LogisticRegression()
lr.fit(X_train_vec, y_train)
y_pred = lr.predict(X_test_vec)

print(classification_report(y_true=y_test, y_pred=y_pred, target_names=newsgroups_train.target_names))

                        precision    recall  f1-score   support

 comp.sys.mac.hardware       0.90      0.92      0.91       385
        comp.windows.x       0.92      0.93      0.92       395
          misc.forsale       0.79      0.89      0.84       390
             rec.autos       0.91      0.88      0.89       396
       rec.motorcycles       0.96      0.94      0.95       398
    rec.sport.baseball       0.94      0.95      0.94       397
      rec.sport.hockey       0.96      0.96      0.96       399
             sci.crypt       0.97      0.90      0.94       396
       sci.electronics       0.78      0.87      0.82       393
               sci.med       0.94      0.87      0.91       396
             sci.space       0.97      0.94      0.95       394
soc.religion.christian       0.94      0.95      0.95       398
    talk.politics.guns       0.95      0.93      0.94       364
 talk.politics.mideast       0.99      0.93      0.96       376

              accuracy                

**Стандартизация**

In [30]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False)
X_train_sc = scaler.fit_transform(X_train_vec)
X_test_sc = scaler.transform(X_test_vec)
X_train_sc

<8227x62970 sparse matrix of type '<class 'numpy.float64'>'
	with 852092 stored elements in Compressed Sparse Row format>

In [31]:
lr = LogisticRegression()
lr.fit(X_train_sc, y_train)
y_pred = lr.predict(X_test_sc)

print(classification_report(y_true=y_test, y_pred=y_pred, target_names=newsgroups_train.target_names))

                        precision    recall  f1-score   support

 comp.sys.mac.hardware       0.86      0.89      0.87       385
        comp.windows.x       0.92      0.90      0.91       395
          misc.forsale       0.78      0.86      0.82       390
             rec.autos       0.91      0.88      0.89       396
       rec.motorcycles       0.93      0.94      0.93       398
    rec.sport.baseball       0.88      0.95      0.91       397
      rec.sport.hockey       0.94      0.96      0.95       399
             sci.crypt       0.94      0.90      0.92       396
       sci.electronics       0.82      0.79      0.81       393
               sci.med       0.90      0.78      0.83       396
             sci.space       0.93      0.88      0.90       394
soc.religion.christian       0.86      0.97      0.91       398
    talk.politics.guns       0.93      0.93      0.93       364
 talk.politics.mideast       0.96      0.90      0.93       376

              accuracy                

C:\Users\TANYA\.conda\envs\env-gpu\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Стемминг 

Воспользуемся стеммингом вместо лемматизации

In [12]:
from nltk.stem import PorterStemmer

def preprocces(X):
  X_proccess = []
  stemmer = PorterStemmer()

  for x in tqdm(X):
    
    x = x.lower()
    x = nltk.word_tokenize(x)
    x = [word for word in x if word.isalnum()]
    x = [stemmer.stem(w) for w in x]
    x = [word for word in x if not word in stop_words]
    X_proccess.append(' '.join(x))
  return X_proccess


X_train_proc = preprocces(X_train)
X_test_proc = preprocces(X_test)

100%|█████████████████████████████████████████████████████████████████████████████| 5477/5477 [00:29<00:00, 183.58it/s]


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train_proc)
X_test_vec = vectorizer.transform(X_test_proc)
print(X_train_vec.shape)
print(X_test_vec.shape)

(8227, 52550)
(5477, 52550)


In [14]:
lr = LogisticRegression()
lr.fit(X_train_vec, y_train)
y_pred = lr.predict(X_test_vec)

print(classification_report(y_true=y_test, y_pred=y_pred, target_names=newsgroups_train.target_names))

                        precision    recall  f1-score   support

 comp.sys.mac.hardware       0.89      0.90      0.89       385
        comp.windows.x       0.92      0.94      0.93       395
          misc.forsale       0.79      0.88      0.83       390
             rec.autos       0.92      0.89      0.90       396
       rec.motorcycles       0.97      0.94      0.95       398
    rec.sport.baseball       0.95      0.94      0.94       397
      rec.sport.hockey       0.96      0.97      0.96       399
             sci.crypt       0.95      0.90      0.92       396
       sci.electronics       0.76      0.85      0.80       393
               sci.med       0.94      0.89      0.91       396
             sci.space       0.97      0.93      0.95       394
soc.religion.christian       0.93      0.95      0.94       398
    talk.politics.guns       0.94      0.93      0.93       364
 talk.politics.mideast       0.99      0.92      0.96       376

              accuracy                